**Logistic regression is applied for classification tasks, using ridge (L2), lasso (L1), and ElasticNet (a mix of L1 and L2) regularization.**


- **Ridge Regression (L2 penalty):** Adds a penalty proportional to the square of the magnitude of the coefficients.

  $$
  R(\beta_j) = \sum_{j}\beta_j^2
  $$

- **Lasso Regression (L1 penalty):** Adds a penalty proportional to the absolute value of the coefficients, which can drive some coefficients exactly to zero, resulting in sparse models.

  $$
  R(\beta_j) = \sum_{j}|\beta_j|
  $$

- **ElasticNet:** Combines both L1 and L2 penalties, controlled by a mixing parameter $\lambda_{\text{mixing}}$.

  $$
  R(\beta_j) = \lambda_{\text{mixing}} \sum_{j}|\beta_j| + (1 - \lambda_{\text{mixing}}) \sum_{j}\beta_j^2
  $$

### **Importing Necessary Packages**

In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import numpy as np
import torch
import matplotlib.pylab as plt
import seaborn as sns
import pandas as pd

### **Loading Data**

In [3]:
df = pd.read_csv('../Data/phishing_train.csv')

### **Splitting Data**

In [4]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1234, stratify=y)

### **Ridge Logistic Regression**

In [5]:
# Creating pipeline to prevent data leakage
pipeline_rr = Pipeline([
    ('scaler', StandardScaler()), # Scaling Data
    ('model', LogisticRegression( # Logistic Regression
        penalty='l2',             # Ridge Regression
        solver='saga',            # Universal Solver for All 3 Types of Regression
        max_iter=5000,            # Maximum number of iterations for solver to converge
        random_state=1234         # Random State for Consistency
    ))
])

# Defining Parameters for Cross Validation
rr_cv_parameters = {
    'model__C': [0.01, 0.1, 1, 10, 100]
}

# Performing Cross Validation with GridSearchCV
rr = GridSearchCV(
    estimator=pipeline_rr,
    param_grid=rr_cv_parameters,
    cv=5, # 5 folds for CV
    scoring='accuracy',
    n_jobs=-1
)
# Fitting Model
rr.fit(X_train, y_train)

# Finding the Best Model
best_rr = rr.best_estimator_

# Predictions with the best performing model
pred_train_rr = best_rr.predict(X_train)
pred_test_rr = best_rr.predict(X_test)

# Performance Metrics
print("Best Ridge Hyperparameters:", rr.best_params_)
print("Train Accuracy:", accuracy_score(y_train, pred_train_rr))
print("Test Accuracy:", accuracy_score(y_test, pred_test_rr))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_test_rr))

Best Ridge Hyperparameters: {'model__C': 0.1}
Train Accuracy: 0.9295944519825117
Test Accuracy: 0.9294436906377205

Confusion Matrix:
 [[ 898   87]
 [  69 1157]]


### **Lasso Logistic Regression**

In [6]:
# Creating pipeline to prevent data leakage
pipe_lr = Pipeline([
    ('scaler', StandardScaler()), # Scaling Data
    ('model', LogisticRegression( # Logistic Regression
        penalty='l1',             # Lasso Regression
        solver='saga',            # Universal Solver for All 3 Types of Regression
        max_iter=5000,            # Maximum number of iterations for solver to converge
        random_state=1234         # Random State for Consistency
    ))
])

# Defining Parameters for Cross Validation
lr_cv_parameters = {
    'model__C': [0.01, 0.1, 1, 10, 100] #C is the inverse regularization strength; smaller = stronger regularization
}

# Performing Cross Validation with GridSearchCV
lr = GridSearchCV(
    estimator=pipe_lr,
    param_grid=lr_cv_parameters,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

# Fitting the Model
lr.fit(X_train, y_train)

# Finding the Best Model
best_lr = lr.best_estimator_

# Predictions with the best performing model
pred_train_lr = best_lr.predict(X_train)
pred_test_lr = best_lr.predict(X_test)

# Performance Metrics
print("Best Lasso Hyperparameters:", lr.best_params_)
print("Train Accuracy:", accuracy_score(y_train, pred_train_lr))
print("Test Accuracy:", accuracy_score(y_test, pred_test_lr))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_test_lr))


Best Lasso Hyperparameters: {'model__C': 0.1}
Train Accuracy: 0.9311020654304236
Test Accuracy: 0.9308005427408412

Confusion Matrix:
 [[ 895   90]
 [  63 1163]]


### **Elastic Net Logistic Regression**

In [7]:
# Creating pipeline to prevent data leakage
pipe_en = Pipeline([
    ('scaler', StandardScaler()), # Scaling Data
    ('model', LogisticRegression( # Logistic Regression
        penalty='elasticnet',     # Elastic Net
        solver='saga',            # Universal Solver for All 3 Types of Regression
        max_iter=10000,           # Maximum number of iterations for solver to converge
        random_state=1234         # Random State for Consistency
    ))
])

# Defining Parameters for Cross Validation
en_cv_parameters = {
    'model__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9], # # Mix of L1 (lasso) and L2 (ridge) regularization
    'model__C': [0.01, 0.1, 1, 10, 100] # C is the inverse regularization strength; smaller = stronger regularization
}

# Performing Cross Validation with GridSearchCV
en = GridSearchCV(
    estimator=pipe_en,
    param_grid=en_cv_parameters,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

# Fitting the Model
en.fit(X_train, y_train)

# Finding the Best Model
best_en = en.best_estimator_

# Predictions with the best performing model
pred_train_en = best_en.predict(X_train)
pred_test_en = best_en.predict(X_test)

# Performance Metrics
print("Best Hyperparameters:", en.best_params_)
print("Train Accuracy:", accuracy_score(y_train, pred_train_en))
print("Test Accuracy:", accuracy_score(y_test, pred_test_en))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_test_en))


Best Hyperparameters: {'model__C': 0.1, 'model__l1_ratio': 0.7}
Train Accuracy: 0.9301974973616765
Test Accuracy: 0.9298959746720941

Confusion Matrix:
 [[ 895   90]
 [  65 1161]]


**Best Model: Logistic Regression with L1 Lasson Regularization**